In [1]:
import pandas as pd
import numpy as np
from dataset import LABELS
from sklearn.preprocessing import LabelEncoder
import random


In [2]:
from dataset import labels_only_detection_training, labels_only_detection_validation, labels_only_detection_full

In [3]:
from models import Transformer

In [4]:
res = pd.DataFrame(columns=['num_heads', 'dropout_rate', 'learning_rate', 'epoch', 'batch', 'num_layer', 'loss_max','accuracy_max','val_loss_max', 'val_accuracy_max'])
# res.columns = ['gru_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max']
for num_heads in [2,4]:
    for num_layer in [2,4]:
        for epoch in [20, 30]:
            for lr in [0.001, 0.01, 0.1]:
                for batch in [64,128]:
                    model = Transformer(num_heads=num_heads, num_layers=num_layer, dropout_rate=0.2, learning_rate=lr, num_classes=10, batch_size=batch, epoch=epoch)
                    model.train(labels_only_detection_training, labels_only_detection_validation)
                    res = np.concatenate((res, pd.DataFrame([[num_heads,0.2, lr, epoch, batch,num_layer, model.history.history['loss'][-1], model.history.history['accuracy'][-1], model.history.history['val_loss'][-1], model.history.history['val_accuracy'][-1]]], 
                                                                            columns=['num_heads', 'dropout_rate', 'learning_rate', 'epoch', 'batch', 'num_layer', 'loss_max','accuracy_max','val_loss_max', 'val_accuracy_max'])))

Epoch 1/20
330/330 [==============================] - ETA: 0s - loss: 1.6999 - accuracy: 0.4385
Epoch 1: val_accuracy improved from -inf to 0.67210, saving model to models\transformer.h5
330/330 [==============================] - 114s 161ms/step - loss: 1.6999 - accuracy: 0.4385 - val_loss: 0.9731 - val_accuracy: 0.6721
Epoch 2/20
330/330 [==============================] - ETA: 0s - loss: 1.0030 - accuracy: 0.6522
Epoch 2: val_accuracy improved from 0.67210 to 0.74117, saving model to models\transformer.h5
330/330 [==============================] - 39s 118ms/step - loss: 1.0030 - accuracy: 0.6522 - val_loss: 0.7559 - val_accuracy: 0.7412
Epoch 3/20
330/330 [==============================] - ETA: 0s - loss: 0.8543 - accuracy: 0.7049
Epoch 3: val_accuracy improved from 0.74117 to 0.75669, saving model to models\transformer.h5
330/330 [==============================] - 38s 115ms/step - loss: 0.8543 - accuracy: 0.7049 - val_loss: 0.7171 - val_accuracy: 0.7567
Epoch 4/20
330/330 [==========

KeyboardInterrupt: 

In [5]:
pd.DataFrame(res,columns=['num_heads', 'dropout_rate', 'learning_rate', 'epoch', 'batch', 'num_layer', 'loss_max','accuracy_max','val_loss_max', 'val_accuracy_max']).to_pickle('results/transformer_wyniki_only_known.pkl')

In [18]:
results = pd.read_pickle('results\\model_transformer_final_version.pkl')

In [19]:
results[results.val_accuracy == results.val_accuracy.max()]

,num_heads,num_layers,dropout_rate,epoch,batch,loss,loss_max,accuracy,accuracy_max,val_loss,val_loss_max,val_accuracy,val_accuracy_max
19,4.0,1.0,0.2,20.0,64.0,0.690125,2.120008,0.790032,0.790722,0.600954,1.342593,0.812886,0.812886


In [22]:
model = Transformer(4, 1, 0.2, 20, 64, 0.001, (39,44), 11, model_path='models\\transformer_mod.h5')

In [24]:
model.train(label_detection_training, label_detection_validation)

Epoch 1/20
106/107 [============================>.] - ETA: 0s - loss: 1.4390 - accuracy: 0.6051
Epoch 1: val_accuracy improved from -inf to 0.62386, saving model to models\transformer_mod.h5
107/107 [==============================] - 7s 64ms/step - loss: 1.4361 - accuracy: 0.6059 - val_loss: 1.2966 - val_accuracy: 0.6239
Epoch 2/20
107/107 [==============================] - ETA: 0s - loss: 1.2717 - accuracy: 0.6354
Epoch 2: val_accuracy improved from 0.62386 to 0.68123, saving model to models\transformer_mod.h5
107/107 [==============================] - 6s 59ms/step - loss: 1.2717 - accuracy: 0.6354 - val_loss: 1.0730 - val_accuracy: 0.6812
Epoch 3/20
107/107 [==============================] - ETA: 0s - loss: 1.1600 - accuracy: 0.6546
Epoch 3: val_accuracy did not improve from 0.68123
107/107 [==============================] - 6s 59ms/step - loss: 1.1600 - accuracy: 0.6546 - val_loss: 1.2423 - val_accuracy: 0.6174
Epoch 4/20
107/107 [==============================] - ETA: 0s - loss: 1.

In [9]:
import tensorflow as tf
import numpy as np
from models import Lstm, Gru, Transformer
from dataset import LABELS
import pandas as pd

In [10]:
test = pd.read_pickle('extracted_features\\features_test.pkl')
dataset = tf.data.Dataset.from_tensor_slices((test, np.zeros(len(test))))


In [ ]:
# save model


In [12]:
x = model.predict(dataset)

15854/15854 [==============================] - 58s 4ms/step


In [14]:
pd.DataFrame(x).value_counts()

10    97594
7      9068
2      8487
5      7625
4      7303
9      6912
0      6628
8      4575
6      4372
1      4179
3      1795
dtype: int64

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dropout, Dense, Bidirectional, TimeDistributed, BatchNormalization
from dataset import LABELS
from sklearn.preprocessing import LabelEncoder
from dataset import label_detection_training, label_detection_validation, silence_detection_training, silence_detection_validation
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical
from dataset import TensorflowDataset

c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# train = pd.read_pickle('extracted_features\\features_training.pkl')
# val = pd.read_pickle('extracted_features\\features_validation.pkl')

# X_train = np.array([x[0] for x in train])
# y_train = np.array([x[1] for x in train])
# X_val = np.array([x[0] for x in val])
# y_val = np.array([x[1] for x in val])

# # Create a label encoder object
# label_encoder = LabelEncoder()

# # Fit the label encoder using your labels (combine both train and val labels if they have different classes)
# label_encoder.fit(np.concatenate((y_train, y_val)))

# # Transform your string labels to integer labels for both training and validation sets
# y_train_encoded = label_encoder.transform(y_train)
# y_val_encoded = label_encoder.transform(y_val)

In [3]:
train = pd.read_pickle('extracted_features\\features_training.pkl')
y_train = np.array([x[1] for x in train])
labels = list(np.unique(y_train))
train = TensorflowDataset('extracted_features\\features_training.pkl', labels=labels).dataset
train = train.shuffle(len(train), reshuffle_each_iteration=True)
val = TensorflowDataset('extracted_features\\features_validation.pkl', labels=labels).dataset
val = val.shuffle(len(val), reshuffle_each_iteration=True)

In [8]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (MultiHeadAttention, Dropout, LayerNormalization, Dense, TimeDistributed,
                                     BatchNormalization, Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D)
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

results = pd.DataFrame(columns=['num_heads', 'num_layers', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, dropout_rate):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential([
            Dense(embed_dim, activation='relu'),
            Dense(embed_dim)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

for num_heads in [2, 4]:
    for num_layers in [1, 2]:
        for dropout_rate in [0.2, 0.4]:
            for batch_size in [32, 64]:
                for epochs in [ 10, 20]:
                    print(f'num_heads: {num_heads}, num_layers: {num_layers}, dropout_rate: {dropout_rate}, batch_size: {batch_size}, epochs: {epochs}')
                    num_classes = len(np.unique(labels))  # Number of unique classes in your dataset
                    input_shape = (39, 44)
                    embed_dim = 128

                    inputs = tf.keras.Input(shape=input_shape)
                    x = Conv1D(filters=128, kernel_size=3, activation='relu')(inputs)
                    x = BatchNormalization()(x)
                    x = MaxPooling1D(pool_size=2)(x)
                    x = Dropout(dropout_rate)(x)

                    for _ in range(num_layers):
                        x = TransformerBlock(embed_dim, num_heads, dropout_rate)(x)

                    x = GlobalMaxPooling1D()(x)
                    x = Dense(512, activation='relu')(x)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)
                    x = Dense(256, activation='relu')(x)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)
                    x = Dense(128, activation='relu')(x)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)
                    outputs = Dense(num_classes, activation='softmax')(x)

                    model = tf.keras.Model(inputs=inputs, outputs=outputs)

                    # Compile the model
                    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

                    # Train the model with your training set and validate it with your validation set
                    history = model.fit(train.batch(batch_size), epochs=epochs, batch_size=batch_size, validation_data=val.batch(batch_size), verbose=2)
                                    # Record the results for the current configuration
                    row = {
                        'num_heads': num_heads,
                        'num_layers': num_layers,
                        'dropout_rate': dropout_rate,
                        'epoch': epochs,
                        'batch': batch_size,
                        'loss': history.history['loss'][-1],
                        'loss_max': np.max(history.history['loss']),
                        'accuracy': history.history['accuracy'][-1],
                        'accuracy_max': np.max(history.history['accuracy']),
                        'val_loss': history.history['val_loss'][-1],
                        'val_loss_max': np.max(history.history['val_loss']),
                        'val_accuracy': history.history['val_accuracy'][-1],
                        'val_accuracy_max': np.max(history.history['val_accuracy']),
                    }

                    results = results.append(row, ignore_index=True)


num_heads: 2, num_layers: 1, dropout_rate: 0.2, batch_size: 32, epochs: 10
Epoch 1/10
1811/1811 - 49s - loss: 2.2341 - accuracy: 0.3708 - val_loss: 1.4084 - val_accuracy: 0.5799 - 49s/epoch - 27ms/step
Epoch 2/10
1811/1811 - 45s - loss: 1.5168 - accuracy: 0.5522 - val_loss: 1.0448 - val_accuracy: 0.6921 - 45s/epoch - 25ms/step
Epoch 3/10
1811/1811 - 48s - loss: 1.3047 - accuracy: 0.6113 - val_loss: 1.1216 - val_accuracy: 0.6602 - 48s/epoch - 27ms/step
Epoch 4/10
1811/1811 - 49s - loss: 1.2991 - accuracy: 0.6142 - val_loss: 0.9259 - val_accuracy: 0.7234 - 49s/epoch - 27ms/step
Epoch 5/10
1811/1811 - 49s - loss: 1.1588 - accuracy: 0.6563 - val_loss: 0.8972 - val_accuracy: 0.7286 - 49s/epoch - 27ms/step
Epoch 6/10
1811/1811 - 48s - loss: 1.1611 - accuracy: 0.6545 - val_loss: 0.8835 - val_accuracy: 0.7343 - 48s/epoch - 26ms/step
Epoch 7/10
1811/1811 - 50s - loss: 1.1062 - accuracy: 0.6720 - val_loss: 0.8617 - val_accuracy: 0.7398 - 50s/epoch - 28ms/step
Epoch 8/10
1811/1811 - 51s - loss: 1

C:\Users\mikol\AppData\Local\Temp\ipykernel_16556\2367907417.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(row, ignore_index=True)


Epoch 1/20
1811/1811 - 66s - loss: 2.1068 - accuracy: 0.4024 - val_loss: 1.2203 - val_accuracy: 0.6320 - 66s/epoch - 37ms/step
Epoch 2/20
1811/1811 - 60s - loss: 1.3975 - accuracy: 0.5842 - val_loss: 1.0267 - val_accuracy: 0.6958 - 60s/epoch - 33ms/step
Epoch 3/20
1811/1811 - 61s - loss: 1.2557 - accuracy: 0.6253 - val_loss: 0.9898 - val_accuracy: 0.7030 - 61s/epoch - 34ms/step
Epoch 4/20
1811/1811 - 60s - loss: 1.1790 - accuracy: 0.6498 - val_loss: 0.9405 - val_accuracy: 0.7162 - 60s/epoch - 33ms/step
Epoch 5/20
1811/1811 - 59s - loss: 1.1051 - accuracy: 0.6706 - val_loss: 0.8465 - val_accuracy: 0.7433 - 59s/epoch - 32ms/step
Epoch 6/20
1811/1811 - 59s - loss: 1.0588 - accuracy: 0.6832 - val_loss: 0.8125 - val_accuracy: 0.7501 - 59s/epoch - 33ms/step
Epoch 7/20
1811/1811 - 59s - loss: 1.0260 - accuracy: 0.6957 - val_loss: 0.7454 - val_accuracy: 0.7695 - 59s/epoch - 33ms/step
Epoch 8/20
1811/1811 - 59s - loss: 1.0234 - accuracy: 0.6941 - val_loss: 0.8140 - val_accuracy: 0.7492 - 59s/ep

C:\Users\mikol\AppData\Local\Temp\ipykernel_16556\2367907417.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(row, ignore_index=True)


Epoch 1/10
906/906 - 69s - loss: 2.1687 - accuracy: 0.3928 - val_loss: 1.2507 - val_accuracy: 0.6231 - 69s/epoch - 76ms/step
Epoch 2/10
906/906 - 59s - loss: 1.3432 - accuracy: 0.6033 - val_loss: 0.9850 - val_accuracy: 0.6986 - 59s/epoch - 65ms/step
Epoch 3/10
906/906 - 59s - loss: 1.1516 - accuracy: 0.6575 - val_loss: 0.9089 - val_accuracy: 0.7239 - 59s/epoch - 65ms/step
Epoch 4/10
906/906 - 50s - loss: 1.0466 - accuracy: 0.6855 - val_loss: 0.8129 - val_accuracy: 0.7557 - 50s/epoch - 55ms/step
Epoch 5/10
906/906 - 59s - loss: 0.9956 - accuracy: 0.7029 - val_loss: 0.8737 - val_accuracy: 0.7315 - 59s/epoch - 65ms/step
Epoch 6/10
906/906 - 60s - loss: 0.9615 - accuracy: 0.7107 - val_loss: 0.8457 - val_accuracy: 0.7429 - 60s/epoch - 67ms/step
Epoch 7/10
906/906 - 58s - loss: 0.9409 - accuracy: 0.7180 - val_loss: 0.7405 - val_accuracy: 0.7764 - 58s/epoch - 64ms/step
Epoch 8/10
906/906 - 61s - loss: 0.8800 - accuracy: 0.7343 - val_loss: 0.7344 - val_accuracy: 0.7732 - 61s/epoch - 67ms/step


...

C:\Users\mikol\AppData\Local\Temp\ipykernel_16556\2367907417.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(row, ignore_index=True)


Epoch 1/10
1811/1811 - 145s - loss: 3.0230 - accuracy: 0.1724 - val_loss: 2.2363 - val_accuracy: 0.3248 - 145s/epoch - 80ms/step
Epoch 2/10
1811/1811 - 144s - loss: 2.1690 - accuracy: 0.3603 - val_loss: 1.7557 - val_accuracy: 0.4821 - 144s/epoch - 79ms/step
Epoch 3/10
1811/1811 - 146s - loss: 1.8697 - accuracy: 0.4496 - val_loss: 1.5132 - val_accuracy: 0.5363 - 146s/epoch - 81ms/step
Epoch 4/10
1811/1811 - 147s - loss: 1.7377 - accuracy: 0.4873 - val_loss: 1.3509 - val_accuracy: 0.5971 - 147s/epoch - 81ms/step
Epoch 5/10
1811/1811 - 137s - loss: 1.6301 - accuracy: 0.5221 - val_loss: 1.2486 - val_accuracy: 0.6305 - 137s/epoch - 75ms/step
Epoch 6/10
1811/1811 - 146s - loss: 1.5492 - accuracy: 0.5439 - val_loss: 1.1941 - val_accuracy: 0.6427 - 146s/epoch - 81ms/step
Epoch 7/10
1811/1811 - 145s - loss: 1.4713 - accuracy: 0.5674 - val_loss: 1.0975 - val_accuracy: 0.6752 - 145s/epoch - 80ms/step
Epoch 8/10
1811/1811 - 144s - loss: 1.4047 - accuracy: 0.5864 - val_loss: 1.1295 - val_accuracy: 

C:\Users\mikol\AppData\Local\Temp\ipykernel_16556\2367907417.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(row, ignore_index=True)


Epoch 1/20
1811/1811 - 143s - loss: 3.0259 - accuracy: 0.1751 - val_loss: 2.1750 - val_accuracy: 0.3420 - 143s/epoch - 79ms/step
Epoch 2/20
1811/1811 - 133s - loss: 2.1249 - accuracy: 0.3700 - val_loss: 1.6614 - val_accuracy: 0.4957 - 133s/epoch - 74ms/step
Epoch 3/20
1811/1811 - 132s - loss: 1.9152 - accuracy: 0.4344 - val_loss: 1.4920 - val_accuracy: 0.5559 - 132s/epoch - 73ms/step
Epoch 4/20
1811/1811 - 132s - loss: 1.7713 - accuracy: 0.4804 - val_loss: 1.3447 - val_accuracy: 0.6025 - 132s/epoch - 73ms/step
Epoch 5/20
1811/1811 - 125s - loss: 1.6620 - accuracy: 0.5112 - val_loss: 1.3313 - val_accuracy: 0.5989 - 125s/epoch - 69ms/step
Epoch 6/20
1811/1811 - 133s - loss: 1.5956 - accuracy: 0.5324 - val_loss: 1.2086 - val_accuracy: 0.6365 - 133s/epoch - 73ms/step
Epoch 7/20
1811/1811 - 132s - loss: 1.5860 - accuracy: 0.5338 - val_loss: 1.2962 - val_accuracy: 0.6162 - 132s/epoch - 73ms/step
Epoch 8/20
1811/1811 - 133s - loss: 1.5435 - accuracy: 0.5471 - val_loss: 1.2003 - val_accuracy: 

KeyboardInterrupt: 

In [9]:
results

,num_heads,num_layers,dropout_rate,epoch,batch,loss,loss_max,accuracy,accuracy_max,val_loss,val_loss_max,val_accuracy,val_accuracy_max
0,2.0,1.0,0.2,10.0,32.0,1.020137,2.234147,0.694923,0.695544,0.749380,1.408424,0.765225,0.765225
1,2.0,1.0,0.2,20.0,32.0,0.835371,2.106825,0.752862,0.752862,0.670023,1.220319,0.798176,0.798176
2,2.0,1.0,0.2,10.0,64.0,0.836061,2.168659,0.748770,0.748770,0.697845,1.250656,0.782142,0.782142
3,2.0,1.0,0.2,20.0,64.0,0.714789,2.103729,0.783126,0.783126,0.629499,1.196326,0.812298,0.820683
4,2.0,1.0,0.4,10.0,32.0,1.258716,2.888041,0.630682,0.630682,1.075352,1.915189,0.681083,0.694322
5,2.0,1.0,0.4,20.0,32.0,1.282760,2.965580,0.632391,0.641438,1.102142,1.910666,0.684025,0.708444
6,2.0,1.0,0.4,10.0,64.0,1.257138,3.018563,0.627350,0.627350,0.929122,1.939173,0.722124,0.722124
7,2.0,1.0,0.4,20.0,64.0,1.190626,3.028895,0.648482,0.671771,0.864657,1.711817,0.743307,0.754340
8,2.0,2.0,0.2,10.0,32.0,0.954365,2.257917,0.715571,0.715571,0.818004,1.401505,0.757134,0.758753
9,2.0,2.0,0.2,20.0,32.0,0.772999,2.242363,0.769142,0.770644,0.697234,1.372575,0.789938,0.798029


In [10]:
results.to_pickle('results\\model_transformer_final_version.pkl')